In [1]:
# Data prep
import sys, os, shutil
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D, MaxPooling2D, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, RandomFlip, RandomZoom, RandomContrast, RandomRotation
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import kerastuner as kt
from random import random
import pandas as pd
import re
import tensorflow_addons as tfa
physical_devices = tf.config.experimental.list_physical_devices('GPU')
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f'Running on Python {sys.version}, Tensorflow {tf.__version__}.')

Running on Python 3.8.10 (tags/v3.8.10:3d8993a, May  3 2021, 11:48:03) [MSC v.1928 64 bit (AMD64)], Tensorflow 2.4.1.


In [2]:
# data loading config
batch_size = 32
img_height = 128
img_width = 128
dataPath = 'oxford_flowers_102'
labels = 'inferred'
label_mode = 'categorical'  # sparse one hot encoding
color_mode = 'rgb'
shuffle = True
seed = 69
test_split = 0.2  # split into train and test (NOT val), 0-1
AUTOTUNE = tf.data.AUTOTUNE

train_df = pd.read_csv('image_to_label.csv', names=['picture', 'label'])
with open('102_flower_labels.txt') as f:
    regex = re.compile('[^a-zA-Z\s-]')
    classes=[]
    for line in f:
        classes += [regex.sub('', line.replace('\'', '').strip())]


def split_data(dataPath):
    for root, dirs, files in os.walk(dataPath):
        for name in files:
            randomNum = random()
            row = train_df.iloc[train_df.index[train_df['picture'] == name]]
            if randomNum <= test_split:
                os.makedirs('test\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'test\\'+classes[row.label.tolist()[0]-1]+'\\')
            elif test_split< randomNum <= test_split + test_split * (1-test_split):
                os.makedirs('val\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'val\\'+classes[row.label.tolist()[0]-1]+'\\')
            else:
                os.makedirs('train\\'+classes[row.label.tolist()[0]-1]+'\\', exist_ok=True)
                shutil.move(root+'\\'+name, 'train\\'+classes[row.label.tolist()[0]-1]+'\\')

# split_data(dataPath)  # Only need to run this once

train_datagen = ImageDataGenerator( # Do 0-1 scaling as a layer so that saved model includes it
    rotation_range=20, width_shift_range=0.2,
    height_shift_range=0.2, brightness_range=(-0.2, 0.2), shear_range=0.2, zoom_range=0.2,
    channel_shift_range=0.2, fill_mode='nearest', horizontal_flip=True, vertical_flip=True)


print('Training data:')
train_generator = train_datagen.flow_from_directory('train', target_size=(img_height, img_width), batch_size=batch_size,
                                                    color_mode=color_mode, class_mode=label_mode, shuffle=shuffle, seed=seed)
train = keras.preprocessing.image_dataset_from_directory('train', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
train_class_names = train.class_names
print('\nValidation data:')
val = keras.preprocessing.image_dataset_from_directory('val', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
val_class_names = val.class_names
print('\nTesting data:')
test = keras.preprocessing.image_dataset_from_directory('test', labels=labels, label_mode=label_mode,
color_mode=color_mode, shuffle=shuffle, seed=seed, image_size=(img_height, img_width), batch_size=batch_size)
test_class_names = test.class_names

train = train.cache().prefetch(buffer_size=AUTOTUNE)
val = val.cache().prefetch(buffer_size=AUTOTUNE)
test = test.cache().prefetch(buffer_size=AUTOTUNE)

assert list(train_generator.class_indices.keys()) == train_class_names == val_class_names == test_class_names, 'Classes mismatch!'
classes = list(train_generator.class_indices.keys())
print('\nClasses:', classes)

Training data:
Found 5248 images belonging to 102 classes.
Found 5248 files belonging to 102 classes.

Validation data:
Found 1320 files belonging to 102 classes.

Testing data:
Found 1621 files belonging to 102 classes.

Classes: ['alpine sea holly', 'anthurium', 'artichoke', 'azalea', 'ball moss', 'balloon flower', 'barbeton daisy', 'bearded iris', 'bee balm', 'bird of paradise', 'bishop of llandaff', 'black-eyed susan', 'blackberry lily', 'blanket flower', 'bolero deep blue', 'bougainvillea', 'bromelia', 'buttercup', 'californian poppy', 'camellia', 'canna lily', 'canterbury bells', 'cape flower', 'carnation', 'cautleya spicata', 'clematis', 'colts foot', 'columbine', 'common dandelion', 'corn poppy', 'cyclamen', 'daffodil', 'desert-rose', 'english marigold', 'fire lily', 'foxglove', 'frangipani', 'fritillary', 'garden phlox', 'gaura', 'gazania', 'geranium', 'giant white arum lily', 'globe thistle', 'globe-flower', 'grape hyacinth', 'great masterwort', 'hard-leaved pocket orchid', '

In [7]:
import functools
from tensorflow.python.ops import nn_ops

class YiqinIsGay(layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1,1), padding='SAME'):
        self.strides = list(strides)
        self.padding = padding.upper()
        self.filters = filters
        self.kernel_size = kernel_size
        super(YiqinIsGay, self).__init__()
        print('init done')

    def build(self, input_shape):
        print('build start')
        kernel_shape = (self.kernel_size, 2) + (input_shape[-1], self.filters)  # (self.kernel_size, 2): 2 for 2D
        print(kernel_shape)
        print('build 1')
        self.kernel = self.add_weight(name='YiqinIsHeavy', shape=kernel_shape, initializer=glorot_uniform)
        print('build 2')
        self.bias = self.add_weight(name='YiqinIsBiased', shape=(self.filters,), initializer=glorot_uniform)
        print('build 3')
        self._convolution_op = functools.partial(nn_ops.convolution_v2, strides=self.strides, padding=self.padding, name='Conv2D')
        print('build 4 (done)')

    def call(self, inputs):
        # print('Yiqin is Gay')
        print('call start')
        self.kernel = self.kernel * tf.math.sqrt(2/(self.kernel_size * self.kernel_size * self.filters))
        print('call 1')
        outputs = self._convolution_op(inputs, self.kernel)
        print('call 2')
        outputs = tf.nn.bias_add(outputs, self.bias)
        print('call 3 (done)')
        return outputs

    def compute_output_shape(self, input_shape):
        print('compute shape start')
        return input_shape[:-1] + (self.filters,)

In [18]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=15, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=7, verbose=1)
]
epoch = 200
def HyperCNN(hyparms=None):
    # tuned_dense_units = hyparms.Int('units', min_value=102, max_value=4096, step=32)
    # tuned_dropout = hyparms.Float('dropout', min_value=0.1, max_value=0.9, step=0.1)
    # tuned_lr = hyparms.Float('learning_rate', min_value=1e-7, max_value=1e-1, step=1e-5)
    # tuned_momentum = hyparms.Float('momentum', min_value=0.1, max_value=0.9, step=0.01)
    # tuned_clipvalue = hyparms.Float('clipvalue', min_value=0.1, max_value=0.9, step=0.1)
    # tuned_epsilon = hyparms.Float('eps', min_value=1e-9, max_value=1e-4, step=1e-7)
    """
    TRASH TUNER SHIT ACC for ADAM
    Best val_accuracy So Far: 0.6151515245437622
    Total elapsed time: 02h 56m 12s
    INFO:tensorflow:Oracle triggered exit
    Dense: 1094, dropout: 0.4, lr: 0.0007001, clipvalue: 0.6, eps: 1.7500999999999997e-05"""

    '''
    For SGD also trash lmao
    Trial 254 Complete [00h 04m 37s]
    val_accuracy: 0.269696980714798

    Best val_accuracy So Far: 0.6689394116401672
    Total elapsed time: 02h 37m 33s
    INFO:tensorflow:Oracle triggered exit
    lr: 0.0163101, momentum: 0.745
    '''

    opt = keras.optimizers.Adam(learning_rate=1e-3, epsilon=1e-6, clipvalue=0.8)  # 1e-3, 1e-6, 0.8
    # opt = keras.optimizers.SGD(learning_rate=0.0163101, momentum=0.745)
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    metrics = ['accuracy' , tfa.metrics.F1Score(num_classes=len(classes))]
    reg = None  # L2 give 74.15

    # Model
    xInput = Input((img_height, img_width, 3))
    x = Rescaling(1./255)(xInput)  # fit 0-255 into 0 and 1
    # x = RandomFlip("horizontal_and_vertical")(x)
    # x = RandomContrast(0.2)(x)
    # x = RandomRotation(0.2)(x)
    # x = RandomZoom(0.3)(x)

    x = Conv2D(32, 3, kernel_regularizer=reg)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(64, 3, kernel_regularizer=reg)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(128, 3, kernel_regularizer=reg)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(256, 3, kernel_regularizer=reg)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Conv2D(512, 3, kernel_regularizer=reg)(x)
    x = BatchNormalization(axis=3)(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=2)(x)

    x = Dropout(0.5)(x)  # 0.5
    x = Flatten()(x)
    x = Dense(512, kernel_regularizer=reg)(x)  # 512
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)  # 0.5
    xOutput = Dense(len(classes), kernel_regularizer=reg)(x)  # no softmax to use logit
    model = keras.Model(xInput, xOutput)
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    model.summary()
    return model

# tuner = kt.Hyperband(HyperCNN, objective='val_accuracy', max_epochs=150, factor=3, directory='SimpleCNN Tuning 2', project_name='Oxford Flowers 102 Simple CNN')
# tuner.search_space_summary()
# tuner.search(train, validation_data=val, epochs=200, callbacks=callbacks, verbose=1)
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
# print(f'lr: {best_hps.get("learning_rate")}, momentum: {best_hps.get("momentum")}')
# print(f'Dense: {best_hps.get("units")}, dropout: {best_hps.get("dropout")}, lr: {best_hps.get("learning_rate")}, clipvalue: {best_hps.get("clipvalue")}, eps: {best_hps.get("eps")}')
model = HyperCNN()
history = model.fit(train, epochs=epoch, validation_data=val, callbacks=callbacks, verbose=1)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
rescaling_5 (Rescaling)      (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_22 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
activation_27 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 61, 61, 64)        1849

In [20]:
# model = keras.models.load_model('oxford_flower_102')
# model.load_weights('SimpleCNN Tuning/Oxford Flowers 102 Simple CNN/trial_655a37da88c679273aa6f9adbace27d5/checkpoints/epoch_0')
model.evaluate(test)
model.save('oxford_flower_102')

51/51 [==============================] - 0s 6ms/step - loss: 1.7020 - accuracy: 0.7415 - f1_score: 0.7141
INFO:tensorflow:Assets written to: oxford_flower_102\assets


In [8]:
def identity_block(X, f, filters, stage, block, reg=None):
    # Conv2D - BN - Relu - Conv2D - BN - Relu - Conv2D - BN - add input - Relu
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    epsilon = 1.001e-5

    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name=bn_name_base + '2c')(X)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

def convolutional_block(X, f, filters, stage, block, s=2, reg=None):
    # input - Conv2D - BN - added
    # Conv2D - BN - Relu - Conv2D - BN - Relu - Conv2D - BN - add input - Relu
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    F1, F2, F3 = filters
    epsilon = 1.001e-5

    X_shortcut = X
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '2a', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name=bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', name=conv_name_base + '2b', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name=bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', name=conv_name_base + '2c', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name=bn_name_base + '2c')(X)

    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', name=conv_name_base + '1', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + '1', epsilon=epsilon)(X_shortcut)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

def augmentation(X):
    X = RandomFlip("horizontal_and_vertical")(X)
    X = RandomContrast(0.2)(X)
    X = RandomRotation(0.2)(X)
    X = RandomZoom(0.2)(X)
    return X

def ResNet50(reg=None, input_shape=(img_height, img_width, 3), classes=len(train_class_names)):
    # Conv2D - BN - Relu - MaxPool - ConvBlock - IDBlock - ConvBlock - IDBlock - IDBlock - IDBlock - ConvBlock
    # - IDBlock - IDBlock - IDBlock - IDBlock - IDBlock - ConvBlock - IDBlock - IDBlock - AvgPool - Flatten - Dense
    stage2f, stage3f, stage4f, stage5f = [64, 64, 256], [128, 128, 512], [256, 256, 1024], [512, 512, 2048]
    epsilon = 1.001e-5

    XInput = Input(input_shape, name='input')
    X = augmentation(XInput)
    # X = Rescaling(1./255)(XInput)  # fit 0-255 into 0 and 1
    X = ZeroPadding2D((3, 3))(X)

    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    X = BatchNormalization(axis=3, epsilon=epsilon, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1, reg=reg)
    X = identity_block(X, 3, stage2f, stage=2, block='b', reg=reg)
    X = identity_block(X, 3, stage2f, stage=2, block='c', reg=reg)
    X = Dropout(0.5)(X)

    X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2, reg=reg)
    X = identity_block(X, 3, stage3f, stage=3, block='b', reg=reg)
    X = identity_block(X, 3, stage3f, stage=3, block='c', reg=reg)
    X = identity_block(X, 3, stage3f, stage=3, block='d', reg=reg)
    X = Dropout(0.5)(X)

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2, reg=reg)
    X = identity_block(X, 3, stage4f, stage=4, block='b', reg=reg)
    X = identity_block(X, 3, stage4f, stage=4, block='c', reg=reg)
    X = identity_block(X, 3, stage4f, stage=4, block='d', reg=reg)
    X = identity_block(X, 3, stage4f, stage=4, block='e', reg=reg)
    X = identity_block(X, 3, stage4f, stage=4, block='f', reg=reg)
    X = Dropout(0.5)(X)

    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2, reg=reg)
    X = identity_block(X, 3, stage5f, stage=5, block='b', reg=reg)
    X = identity_block(X, 3, stage5f, stage=5, block='c', reg=reg)

    X = GlobalAveragePooling2D()(X)
    X = Dropout(0.5)(X)
    X = Flatten()(X)
    XOutput = Dense(classes, name='final_dense', activation='softmax', kernel_initializer=glorot_uniform(seed=0), kernel_regularizer=reg)(X)
    return keras.Model(XInput, XOutput, name='ResNet50')

In [9]:
opt = keras.optimizers.Adam(learning_rate=3e-4, epsilon=1e-6)
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
reg = keras.regularizers.L2(1e-5)
epoch = 200
metrics = ['accuracy' , tfa.metrics.F1Score(num_classes=len(classes))]
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1,
                                     mode='auto', baseline=None, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('./best_model',monitor='val_accuracy',save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=3, verbose=1)
]

model = ResNet50(reg=reg, input_shape=(img_height, img_width, 3), classes=len(train_class_names))
model.compile(optimizer=opt, loss=loss, metrics=metrics)
model.summary()
history = model.fit(train, epochs=epoch, validation_data=val, callbacks=callbacks, verbose=1)

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
rescaling_1 (Rescaling)         (None, 128, 128, 3)  0           input[0][0]                      
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 134, 134, 3)  0           rescaling_1[0][0]                
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 64, 64, 64)   9472        zero_padding2d_1[0][0]           
___________________________________________________________________________________________

In [10]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True, to_file='model.png')
model.evaluate(test)
model.save('oxford_flower_102')


51/51 [==============================] - 1s 22ms/step - loss: 2.0353 - accuracy: 0.5379 - f1_score: 0.4955